# 基于新闻文档的方案 - 本地嵌入模型 gte-qwen2-1.5b

初步结论：

- 使用本地嵌入模型（gte-qwen2-1.5b），目前测试的召回率最高的嵌入模型
- k=1时，准确率在 `83.3%`

如果使用单一召回方式，可以用在一些适当的项目里了。

后续可以考虑增加检索方式，比如基于关键字


## 准备

In [1]:
%%time

INDEX_PATH="news-index-local"
DATA_PATH="news-data-local"

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 3.1 µs


In [2]:
%%time
!rm -rf $INDEX_PATH

CPU times: user 6.32 ms, sys: 201 µs, total: 6.53 ms
Wall time: 109 ms


In [3]:
%%time

news_urls=[
    "https://www.guancha.cn/internation/2024_06_26_739376_s.shtml",
    "https://user.guancha.cn/main/content?id=1257049",
    "https://user.guancha.cn/main/content?id=1257033",
    "https://www.guancha.cn/politics/2024_06_26_739354.shtml",
    "https://www.guancha.cn/gaotianwei/2024_06_24_739062_s.shtml",
    "https://www.guancha.cn/economy/2024_06_26_739352.shtml"
]

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 3.34 µs


In [4]:
%%time

import requests
from gne import GeneralNewsExtractor

def get_news_data(url):
    response = requests.get(url)
    html = response.text

    extractor = GeneralNewsExtractor()
    data = extractor.extract(html, noise_node_list=[
                               '//div[@class="comment-list"]'])
    data['url']=url
    return data

CPU times: user 115 ms, sys: 70.4 ms, total: 186 ms
Wall time: 85.8 ms


In [6]:
%%time

!mkdir -p $DATA_PATH

import json

for news_url in news_urls:
    data=get_news_data(news_url)
    data['url']=news_url
    # a.append(data)
    # data_dict = json.loads(data)
    file_path = f'./{DATA_PATH}/{data["title"]}.json'
    with open(file_path, 'w') as json_file:
        json.dump(data, json_file, indent=4)

CPU times: user 678 ms, sys: 20.8 ms, total: 698 ms
Wall time: 1.68 s


In [7]:
%%time

from llama_index.core import(
    Document
)

def data2doc(news_data):
    document=Document(text=news_data['content'], 
                  metadata={"title": news_data['title'],
                            'publish_time': news_data['publish_time'],
                            'author': news_data['author'],
                            'url': news_data['url'],
                            'images': news_data['images'],
                           })
    document.doc_id = document.metadata["title"]
    return document

CPU times: user 2.5 s, sys: 220 ms, total: 2.72 s
Wall time: 2.53 s


In [8]:
%%time

from llama_index.core import SimpleDirectoryReader

documents=SimpleDirectoryReader(input_dir="./news-data").load_data(num_workers=4)
for document in documents:
    document.doc_id=document.metadata['file_name']

CPU times: user 17.3 ms, sys: 562 µs, total: 17.9 ms
Wall time: 3.24 s


In [9]:
%%time

import json

docs=[]
for document in documents:
    news_data=json.loads(document.text) # documents[0].text
    docs.append(data2doc(news_data))

documents=docs

len(docs)

CPU times: user 794 µs, sys: 0 ns, total: 794 µs
Wall time: 753 µs


6

In [10]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 720 µs, sys: 76 µs, total: 796 µs
Wall time: 783 µs


In [11]:
%%time

# 加载llm和embeddings
%run ../utils2.py

from llama_index.core import Settings


Settings.llm=get_llm() 
Settings.embed_model=get_embedding("rjmalagon/gte-qwen2-1.5b-instruct-embed-f16")

CPU times: user 689 ms, sys: 19.5 ms, total: 709 ms
Wall time: 709 ms


In [12]:
%%time

from llama_index.core import get_response_synthesizer
from llama_index.core import DocumentSummaryIndex
from llama_index.core.node_parser import SentenceSplitter

splitter = SentenceSplitter(chunk_size=1024)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", use_async=True
)

doc_summary_index = DocumentSummaryIndex.from_documents(
    documents,
    transformations=[splitter],
    response_synthesizer=response_synthesizer,
    show_progress=True,
)

Parsing nodes:   0%|          | 0/6 [00:00<?, ?it/s]

Summarizing documents:   0%|          | 0/6 [00:00<?, ?it/s]

current doc id: 丁刚：中印能实现龙象共舞吗？
current doc id: 中小银行改革加速度，今年来超80家开启合并重组模式
current doc id: 印度总理莫迪计划访问俄罗斯
current doc id: 金融监管总局：将研究提高保险资金投资创业投资基金集中度的比例上限
current doc id: 韩国电池厂火灾17名中国人遇难：有多少同胞，在韩国打工？
current doc id: 高天伟：国产可重复火箭完成10公里测试，有一点与马斯克很不一样


Generating embeddings:   0%|          | 0/6 [00:00<?, ?it/s]

CPU times: user 405 ms, sys: 31.3 ms, total: 436 ms
Wall time: 1min 33s


In [13]:
%%time

doc_summary_index.storage_context.persist(INDEX_PATH)

CPU times: user 30 ms, sys: 0 ns, total: 30 ms
Wall time: 29.3 ms


## 加载索引

In [7]:
%%time

from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

# rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=INDEX_PATH)
doc_summary_index = load_index_from_storage(storage_context)

CPU times: user 28.2 ms, sys: 7.51 ms, total: 35.7 ms
Wall time: 35.3 ms


## 创建查询

### 基于嵌入

In [23]:
%%time

from llama_index.core.indices.document_summary import (
    DocumentSummaryIndexEmbeddingRetriever,
)
from llama_index.core.query_engine import RetrieverQueryEngine

retriever = DocumentSummaryIndexEmbeddingRetriever(
    doc_summary_index,
    # similarity_top_k=3,
    similarity_top_k=1,
)

response_synthesizer = get_response_synthesizer(
    response_mode="tree_summarize", 
    use_async=True,
    streaming=True
)

query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

CPU times: user 158 µs, sys: 0 ns, total: 158 µs
Wall time: 162 µs


### 基于 LLM

In [115]:
%%time

# from llama_index.core.indices.document_summary import (
#     DocumentSummaryIndexLLMRetriever,
# )

# retriever = DocumentSummaryIndexLLMRetriever(
#     doc_summary_index,
#     # choice_select_prompt=None,
#     choice_batch_size=20,
#     # choice_top_k=10,
#     # format_node_batch_fn=None,
#     # parse_choice_select_answer_fn=None,
# )

# response_synthesizer = get_response_synthesizer(
#     response_mode="tree_summarize", 
#     use_async=True,
#     streaming=True
# )

# query_engine = RetrieverQueryEngine(
#     retriever=retriever,
#     response_synthesizer=response_synthesizer,
# )

CPU times: user 131 µs, sys: 0 ns, total: 131 µs
Wall time: 134 µs


## 查询

In [24]:
%%time

response = query_engine.query("莫迪计划访问俄罗斯有哪些重大意义？")
response.print_response_stream()

莫迪计划访问俄罗斯具有多重重大意义：

1. **地缘政治关系的强化**：印度总理纳伦德拉·莫迪与俄罗斯总统弗拉基米尔·普京一直保持着定期沟通，这表明两国在国际事务中有着紧密的合作。通过此次访问，双方可以进一步巩固和深化双边关系。

2. **贸易合作的加强**：自俄乌战争爆发以来，印度作为俄罗斯的主要贸易伙伴的重要性有所增加。印度已成为俄罗斯石油的主要买家之一，在美国及其盟友实施制裁、关闭俄罗斯向西方出口渠道的情况下，这种合作关系变得尤为重要。

3. **经济与商业交流**：两国之间的贸易额在2024财年达到了657亿美元，比上一年增长了33%。尽管近年来印度对俄罗斯的出口并未出现大幅增长，导致了俄罗斯对印度的巨大贸易顺差，但双方目标是实现双边贸易的多元化，并纠正这种不平衡。

4. **外交策略与政策**：在俄乌冲突背景下，印度选择不谴责俄罗斯，而是坚持认为解决方案需要冲突双方进行真诚和务实的接触。这反映了印度在处理国际事务时采取独立立场的做法，同时避免成为西方国家的压力目标。

5. **多边组织内的合作**：作为上合组织、金砖国家成员以及与美国、日本和澳大利亚组成的四方安全组织（QUAD）的一员，莫迪访问俄罗斯也体现了印度在全球政治舞台上的多边外交努力。通过与俄罗斯的合作，印度在国际事务中寻求平衡各方利益。

6. **战略伙伴关系的维护**：莫迪与普京之间的会晤对于消除任何可能阻碍双边关系发展的障碍至关重要，包括解决付款问题、军事技术合作以及扩大经济联系和引入新合作领域等方面。

综上所述，莫迪访问俄罗斯不仅加强了两国在政治、经济和外交层面上的合作，还体现了印度在全球地缘政治格局中的独立性和多边主义立场。CPU times: user 500 ms, sys: 55.3 ms, total: 555 ms
Wall time: 11.2 s


In [25]:
%%time

response = query_engine.query("有多少同胞在韩国打工？")
response.print_response_stream()

根据提供的信息，有超过94万的中国人居住在韩国，并且其中约37.2万人在韩国就业。这表明，在韩国打工的中国同胞数量相当多。CPU times: user 79.3 ms, sys: 8.11 ms, total: 87.4 ms
Wall time: 7.82 s


In [26]:
%%time

response = query_engine.query("韩国电池厂火灾有哪些信息？")
response.print_response_stream()

韩国电池厂火灾的信息包括以下几点：

1. **时间与地点**：事件发生在2024年6月24日，位于京畿道华城市的一家电池工厂。
2. **参与救援的规模**：有160名消防员和60台消防车辆参与扑救工作，最终在8小时后控制了火势。
3. **损失情况**：确认有23人死亡，其中17人为中国籍、5人为韩国籍（含1名中国人加入韩国国籍）以及1名老挝籍人士。
4. **事故工厂背景**：该电池工厂属于韩国锂电池巨头Aricell，共有3层楼。
5. **灾难规模与影响**：被描述为“韩国史上最严重化学工厂惨剧”。
6. **遇难者身份**：大部分是来自中国东北地区的朝鲜族女性，年龄在20岁左右（1999年生、2001年生）。
7. **劳动力需求背景**：韩国面临人口结构挑战和劳动力短缺问题，总和生育率低至全球倒数第一。新生儿数量减少导致人口快速萎缩，为解决劳动力问题，韩国放宽了外国劳工尤其是体力劳动者的就业门槛。

这些信息揭示了火灾事件的严重性、遇难者身份以及韩国在应对人口老龄化和劳动力短缺时面临的挑战。CPU times: user 384 ms, sys: 38.8 ms, total: 423 ms
Wall time: 15.7 s


In [27]:
%%time

response = query_engine.query("国产可重复使用火箭有啥进展？")
response.print_response_stream()

国产可重复使用火箭在多个关键领域取得了显著进展。首先，在技术验证方面，中国航天科技集团八院成功研发的重复使用运载火箭新技术验证箭，在酒泉卫星发射中心完成了10公里级别的飞行试验，这标志着该技术领域的重要突破，并为2025年实现4米级重复使用运载火箭首飞奠定了坚实的技术基础。

其次，民营企业的参与也展现了这一领域的活力。蓝箭航天的朱雀三号VTVL-1可重复使用垂直起降回收验证火箭在首次试飞中即成功完成了300米级别的飞行，并计划近期进行更高水平的10公里级别试验。这表明民营企业在国产可重复使用火箭技术上取得了快速进展。

在燃料技术方面，70吨推力的发液氧甲烷发动机被应用于这些火箭中。这种发动机被认为符合可复用火箭的理想推力区间，“液氧甲烷”也被国际认可为未来清洁可复用燃料的首选。航天八院通过外采关键产品来优化性能、时间和成本，体现了其在商业航天领域的效率优先策略。

供应链管理方面，国产可重复使用火箭展示了自主研制与集成研制的成功结合。例如，民营天兵科技研发的天龙二号火箭首飞成功，并采用了由航天科技集团提供的液发102发动机。这一案例说明了我国商业航天能够充分利用国家工业门类齐全的优势，在供应链优化和成果分享方面取得了进展。

综上所述，国产可重复使用火箭在技术验证、民营企业参与、燃料选择以及供应链管理等方面均展现出积极的进展，为未来投入实用奠定了坚实的基础。CPU times: user 467 ms, sys: 43.8 ms, total: 510 ms
Wall time: 20.8 s


In [28]:
%%time

response = query_engine.query("将研究提高保险资金投资创业投资基金集中度的比例上限主要讲了啥？")
response.print_response_stream()

金融监管总局计划研究并提升保险资金投资于创业投资基金时的集中度比例限制。这一举措旨在更好地引导保险资金和相关资产管理机构，在依法合规、风险可控的前提下，增加对创业投资基金的投资配置力度。通过此举，金融监管总局希望促进创业投资行业的高质量发展，并进一步支持科技创新领域。CPU times: user 91.9 ms, sys: 16 ms, total: 108 ms
Wall time: 2.6 s


In [29]:
%%time

response = query_engine.query("印度总理莫迪计划在赢得第三个任期后做些啥？")
response.print_response_stream()

根据俄罗斯塔斯社的报道，印度总理纳伦德拉·莫迪（Narendra Modi）计划访问俄罗斯。这次访问可能持续两天，并且旨在促进印度与俄罗斯之间的关系。莫迪此行意义重大，因为他在今年的选举中获胜并连任。在之前的访问中，莫迪曾于2019年在远东地区的符拉迪沃斯托克参加经济论坛时访问过俄罗斯，上一次访问莫斯科是在2015年。此外，普京和莫迪一直保持着定期沟通，并在多个场合进行会面。

印度总理莫迪计划访问俄罗斯的背景是，在美国及其盟友对俄罗斯实施制裁后，印度成为了俄罗斯石油的主要买家之一。这表明印度与俄罗斯的关系仍然非常重要，尤其是在当前地缘政治复杂性下，双边关系保持良好状态。尽管西方国家对俄罗斯施加压力，并在乌克兰问题上采取了谴责立场，但印度一直反对单边制裁，并表示不会成为西方国家的目标。

印度与俄罗斯之间的贸易额在2024财年达到了657亿美元，比上一年增长了33%。然而，近年来印度对俄罗斯的出口并未出现大幅增长，导致了俄罗斯对印度的巨大贸易顺差。为了实现双边贸易的多元化并纠正这种不平衡，双方的目标是实现贸易多样化。

综上所述，莫迪计划访问俄罗斯旨在加强两国关系，并可能涉及经济、军事和技术领域的合作。同时，印度在处理与俄罗斯的关系时，也考虑到了来自西方的压力和乌克兰问题上的立场。CPU times: user 409 ms, sys: 55.1 ms, total: 464 ms
Wall time: 9.27 s


In [30]:
len(response.source_nodes)

3

In [31]:
response.source_nodes[1].metadata

{'title': '印度总理莫迪计划访问俄罗斯',
 'publish_time': '2024-06-26 18:54:14',
 'author': '林兆楠',
 'url': 'https://www.guancha.cn/internation/2024_06_26_739376_s.shtml',
 'images': ['https://i.guancha.cn/news/dfic/2024/06/26/20240626173214826.jpg',
  'https://i.guancha.cn/news/external/2024/06/26/20240626172837420.png']}